In [1]:
from bs4 import BeautifulSoup
import urllib.request as ul2
import re, time, random, os, traceback
import pandas as pd
from datetime import datetime

/Users/romanzubarev/miniconda3/envs/nba/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
teams = ['TOR','MIL','PHI','DET','IND','BOS','ORL','CHO','MIA','WAS','BRK','NYK','ATL','CHI','CLE','LAC','DEN','OKC','GSW','LAL','MEM','POR','DAL','NOP','SAC','HOU','SAS','MIN','UTA','PHO']
years = range(2013,datetime.now().year+1)

In [3]:
def fetch_html():
    header  = None
    log     = []
    for y in years:
        for t in teams:
            proceed = True
            filepath = 'html/'+str(y)+'_'+t+'.txt'
            if not os.path.isfile(filepath):
                target_url = 'https://www.basketball-reference.com/teams/'+t+'/'+str(y)+'/gamelog/'
                try:
                    response = ul2.urlopen(target_url).read()
                except Exception:
                    log.append('generic exception: ' + traceback.format_exc())
                    proceed = False
                if proceed:
                    page = BeautifulSoup(response)
                    table = page.find('table', class_='row_summable')
                    if header is None:
                        header = get_header(table)
                    with open(filepath, 'w') as file:
                        file.write(str(table))
                time.sleep(random.randint(4,6))
    return(log, header)

In [4]:
def get_header(table):
    src_header = table.find_all('tr')[1]
    target_header = list()
    for i in src_header.find_all('th'):
        target_header.append(i.attrs['aria-label'])
    target_header[3] = 'Home Court'
    del target_header[24]
    return(target_header[1:])

In [24]:
def get_data(table, team, year):
    data = list()
    for i in table.findAll("tr", id=re.compile("^tgl_basic.")):
        row = list()
        for n in i.find_all('td'):
            row.append(n.string)
        row.append(team)
        row.append(year)
        del row[23]
        data.append(row)
        
    df = pd.DataFrame(data, columns=header)
    df['Home Court'] = df['Home Court'].map(lambda x: 0 if x == '@' else 1)
    df['W/L'] = df['W/L'].map(lambda x: 0 if x == 'L' else 1)
    return(df)

In [25]:
def process_html():
    complete = None
    for y in years:
        for t in teams:
            with open('html/'+str(y)+'_'+t+'.txt', 'r') as file:
                table = BeautifulSoup(file.read())
            data = get_data(table, t, y)
            if complete is None:
                complete = pd.DataFrame(columns=data.columns)
            complete = complete.append(data, ignore_index=True)
    return(complete)

In [73]:
# initial scraping pipeline
log, header = fetch_html()

In [13]:
header = pd.read_csv('header.csv', header=0, quotechar='\'').columns

In [14]:
header

Index(['Season Game', 'Date', 'Home Court', 'Opponent', 'W/L', 'Points',
       'Opponent Points', 'Field Goals', 'Field Goal Attempts',
       'Field Goal Percentage', '3-Point Field Goals',
       '3-Point Field Goal Attempts', '3-Point Field Goal Percentage',
       'Free Throws', 'Free Throw Attempts', 'Free Throw Percentage',
       'Offensive Rebounds', 'Total Rebounds', 'Assists', 'Steals', 'Blocks',
       'Turnovers', 'Personal Fouls', 'Opponent Field Goals',
       'Opponent Field Goal Attempts', 'Opponent Field Goal Percentage',
       'Opponent 3-Point Field Goals', 'Opponent 3-Point Field Goal Attempts',
       'Opponent 3-Point Field Goal Percentage', 'Opponent Free Throws',
       'Opponent Free Throw Attempts', 'Opponent Free Throw Percentage',
       'Opponent Offensive Rebounds', 'Opponent Total Rebounds',
       'Opponent Assists', 'Opponent Steals', 'Opponent Blocks',
       'Opponent Turnovers', 'Opponent Personal Fouls', 'Team', 'Year'],
      dtype='object')

In [26]:
df = process_html()

In [29]:
df.head()

,Season Game,Date,Home Court,Opponent,W/L,Points,Opponent Points,Field Goals,Field Goal Attempts,Field Goal Percentage,...,Opponent Free Throw Percentage,Opponent Offensive Rebounds,Opponent Total Rebounds,Opponent Assists,Opponent Steals,Opponent Blocks,Opponent Turnovers,Opponent Personal Fouls,Team,Year
0,1,2012-10-31,1,IND,0,88,90,33,91,.363,...,.688,9,46,22,3,10,18,16,TOR,2013
1,2,2012-11-03,0,BRK,0,100,107,37,82,.451,...,.730,9,41,20,6,5,17,19,TOR,2013
2,3,2012-11-04,1,MIN,1,105,86,40,90,.444,...,.710,8,36,18,8,8,24,17,TOR,2013
3,4,2012-11-06,0,OKC,0,88,108,30,84,.357,...,.853,7,46,24,10,9,17,22,TOR,2013
4,5,2012-11-07,0,DAL,0,104,109,36,81,.444,...,.657,16,47,21,4,6,11,23,TOR,2013


In [28]:
df.to_csv('all_games_2013_2018.csv', index=False)

In [22]:
from custom_functions import CustomFunctions

In [ ]:
cf = CustomFunctions()
cf.dbconnect()